### Procecdure followed:
1. Clustering is performed on columns 'groups','keywords','topics' after dividing these columns into individual word columns and converting them into dummy variables.
2. The effect of addition of first, second and third words in 'topics' column on the cluster centers is evaluated.

In [148]:
#loading required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [149]:
df=pd.read_csv('[UCI] AAAI-14 Accepted Papers - Papers.csv') #reading data

In [150]:
df.head()

,title,authors,groups,keywords,topics,abstract
0,Kernelized Bayesian Transfer Learning,Mehmet Gönen and Adam A. Margolin,Novel Machine Learning Algorithms (NMLA),cross-domain learning\ndomain adaptation\nkern...,APP: Biomedical / Bioinformatics\nNMLA: Bayesi...,Transfer learning considers related but distin...
1,"""Source Free"" Transfer Learning for Text Class...","Zhongqi Lu, Yin Zhu, Sinno Pan, Evan Xiang, Yu...",AI and the Web (AIW)\nNovel Machine Learning A...,Transfer Learning\nAuxiliary Data Retrieval\nT...,AIW: Knowledge acquisition from the web\nAIW: ...,Transfer learning uses relevant auxiliary data...
2,A Generalization of Probabilistic Serial to Ra...,Haris Aziz and Paul Stursberg,Game Theory and Economic Paradigms (GTEP),social choice theory\nvoting\nfair division\ns...,GTEP: Game Theory\nGTEP: Social Choice / Voting,The probabilistic serial (PS) rule is one of t...
3,Lifetime Lexical Variation in Social Media,"Liao Lizi, Jing Jiang, Ying Ding, Heyan Huang ...",NLP and Text Mining (NLPTM),Generative model\nSocial Networks\nAge Prediction,AIW: Web personalization and user modeling\nNL...,As the rapid growth of online social media att...
4,Hybrid Singular Value Thresholding for Tensor ...,"Xiaoqin Zhang, Zhengyuan Zhou, Di Wang and Yi Ma",Knowledge Representation and Reasoning (KRR)\n...,tensor completion\nlow-rank recovery\nhybrid s...,KRR: Knowledge Representation (General/Other)\...,"In this paper, we study the low-rank tensor co..."


In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 6 columns):
title       398 non-null object
authors     398 non-null object
groups      396 non-null object
keywords    398 non-null object
topics      394 non-null object
abstract    398 non-null object
dtypes: object(6)
memory usage: 18.7+ KB


#### Splitting the groups, keywords and topics data into individual words. Only first 2 words of groups column and first 3 words each of keywords and topics are chosen for easier analysis

In [152]:
df['group1'], df['group2'] = df['groups'].str.split('and', 1).str

In [153]:
df['words1'],df['words2'],df['words3']= df['keywords'].str.split('\n', 2).str

In [154]:
df['topics1'],df['topics2'],df['topics3']=df['topics'].str.split('\n', 2).str

In [155]:
df.head()

,title,authors,groups,keywords,topics,abstract,group1,group2,words1,words2,words3,topics1,topics2,topics3
0,Kernelized Bayesian Transfer Learning,Mehmet Gönen and Adam A. Margolin,Novel Machine Learning Algorithms (NMLA),cross-domain learning\ndomain adaptation\nkern...,APP: Biomedical / Bioinformatics\nNMLA: Bayesi...,Transfer learning considers related but distin...,Novel Machine Learning Algorithms (NMLA),NaN,cross-domain learning,domain adaptation,kernel methods\ntransfer learning\nvariational...,APP: Biomedical / Bioinformatics,NMLA: Bayesian Learning,"NMLA: Kernel Methods\nNMLA: Transfer, Adaptati..."
1,"""Source Free"" Transfer Learning for Text Class...","Zhongqi Lu, Yin Zhu, Sinno Pan, Evan Xiang, Yu...",AI and the Web (AIW)\nNovel Machine Learning A...,Transfer Learning\nAuxiliary Data Retrieval\nT...,AIW: Knowledge acquisition from the web\nAIW: ...,Transfer learning uses relevant auxiliary data...,AI,the Web (AIW)\nNovel Machine Learning Algorit...,Transfer Learning,Auxiliary Data Retrieval,Text Classification,AIW: Knowledge acquisition from the web,AIW: Machine learning and the web,"NMLA: Transfer, Adaptation, Multitask Learning"
2,A Generalization of Probabilistic Serial to Ra...,Haris Aziz and Paul Stursberg,Game Theory and Economic Paradigms (GTEP),social choice theory\nvoting\nfair division\ns...,GTEP: Game Theory\nGTEP: Social Choice / Voting,The probabilistic serial (PS) rule is one of t...,Game Theory,Economic Paradigms (GTEP),social choice theory,voting,fair division\nsocial decision schemes,GTEP: Game Theory,GTEP: Social Choice / Voting,NaN
3,Lifetime Lexical Variation in Social Media,"Liao Lizi, Jing Jiang, Ying Ding, Heyan Huang ...",NLP and Text Mining (NLPTM),Generative model\nSocial Networks\nAge Prediction,AIW: Web personalization and user modeling\nNL...,As the rapid growth of online social media att...,NLP,Text Mining (NLPTM),Generative model,Social Networks,Age Prediction,AIW: Web personalization and user modeling,NLPTM: Information Extraction,NLPTM: Natural Language Processing (General/Ot...
4,Hybrid Singular Value Thresholding for Tensor ...,"Xiaoqin Zhang, Zhengyuan Zhou, Di Wang and Yi Ma",Knowledge Representation and Reasoning (KRR)\n...,tensor completion\nlow-rank recovery\nhybrid s...,KRR: Knowledge Representation (General/Other)\...,"In this paper, we study the low-rank tensor co...",Knowledge Representation,Reasoning (KRR)\nMachine Learning Application...,tensor completion,low-rank recovery,hybrid singular value thresholding,KRR: Knowledge Representation (General/Other),MLA: Machine Learning Applications (General/ot...,NMLA: Data Mining and Knowledge Discovery\nNML...


In [156]:
data=df.drop(['title', 'authors','groups','keywords','topics','abstract',
              'group2','words2','words3','topics2','topics3'], axis=1)

In [157]:
data.head()

,group1,words1,topics1
0,Novel Machine Learning Algorithms (NMLA),cross-domain learning,APP: Biomedical / Bioinformatics
1,AI,Transfer Learning,AIW: Knowledge acquisition from the web
2,Game Theory,social choice theory,GTEP: Game Theory
3,NLP,Generative model,AIW: Web personalization and user modeling
4,Knowledge Representation,tensor completion,KRR: Knowledge Representation (General/Other)


In [158]:
data=pd.get_dummies(data)

In [159]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Columns: 562 entries, group1_AI  to topics1_VIS: Videos
dtypes: uint8(562)
memory usage: 218.5 KB


## Cluster 1

In [160]:
#cluster centers for only first word form each 'groups','keywords' and 'topics' columns
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(data)
kmeans.cluster_centers_

array([[1.63879599e-01, 1.67224080e-02, 1.00334448e-02, ...,
        6.68896321e-03, 3.34448161e-03, 3.34448161e-03],
       [1.38777878e-16, 0.00000000e+00, 5.20417043e-18, ...,
        3.46944695e-18, 1.73472348e-18, 1.73472348e-18],
       [1.24900090e-16, 0.00000000e+00, 4.33680869e-18, ...,
        4.33680869e-18, 2.16840434e-18, 2.16840434e-18]])

In [161]:
data1=df.drop(['title', 'authors','groups','keywords','topics','abstract'], axis=1)
data1.head()

,group1,group2,words1,words2,words3,topics1,topics2,topics3
0,Novel Machine Learning Algorithms (NMLA),NaN,cross-domain learning,domain adaptation,kernel methods\ntransfer learning\nvariational...,APP: Biomedical / Bioinformatics,NMLA: Bayesian Learning,"NMLA: Kernel Methods\nNMLA: Transfer, Adaptati..."
1,AI,the Web (AIW)\nNovel Machine Learning Algorit...,Transfer Learning,Auxiliary Data Retrieval,Text Classification,AIW: Knowledge acquisition from the web,AIW: Machine learning and the web,"NMLA: Transfer, Adaptation, Multitask Learning"
2,Game Theory,Economic Paradigms (GTEP),social choice theory,voting,fair division\nsocial decision schemes,GTEP: Game Theory,GTEP: Social Choice / Voting,NaN
3,NLP,Text Mining (NLPTM),Generative model,Social Networks,Age Prediction,AIW: Web personalization and user modeling,NLPTM: Information Extraction,NLPTM: Natural Language Processing (General/Ot...
4,Knowledge Representation,Reasoning (KRR)\nMachine Learning Application...,tensor completion,low-rank recovery,hybrid singular value thresholding,KRR: Knowledge Representation (General/Other),MLA: Machine Learning Applications (General/ot...,NMLA: Data Mining and Knowledge Discovery\nNML...


In [162]:
data1=pd.get_dummies(data1)

## Cluster 2

In [163]:
#cluster centers for first 2 words from 'groups' and first 3 words from each of 'topics' and 'keywords' columns
kmeans_1 = KMeans(n_clusters=3)
kmeans_1.fit(data1)
kmeans_1.cluster_centers_

array([[ 1.47147147e-01,  1.50150150e-02,  9.00900901e-03, ...,
         6.00600601e-03,  6.00600601e-03,  3.00300300e-03],
       [ 1.24900090e-16,  0.00000000e+00,  4.33680869e-18, ...,
         4.33680869e-18,  4.33680869e-18,  2.16840434e-18],
       [-1.38777878e-17,  0.00000000e+00, -1.73472348e-18, ...,
         8.67361738e-19,  8.67361738e-19,  4.33680869e-19]])

In [164]:
data2=df.drop(['title', 'authors','groups','keywords','topics','abstract',
              'group2','words2','words3','topics3'], axis=1)
data2.head()

,group1,words1,topics1,topics2
0,Novel Machine Learning Algorithms (NMLA),cross-domain learning,APP: Biomedical / Bioinformatics,NMLA: Bayesian Learning
1,AI,Transfer Learning,AIW: Knowledge acquisition from the web,AIW: Machine learning and the web
2,Game Theory,social choice theory,GTEP: Game Theory,GTEP: Social Choice / Voting
3,NLP,Generative model,AIW: Web personalization and user modeling,NLPTM: Information Extraction
4,Knowledge Representation,tensor completion,KRR: Knowledge Representation (General/Other),MLA: Machine Learning Applications (General/ot...


## Cluster 3

In [167]:
data2=pd.get_dummies(data2)
kmeans_2 = KMeans(n_clusters=3)
kmeans_2.fit(data2)
kmeans_2.cluster_centers_

array([[1.63879599e-01, 1.67224080e-02, 1.00334448e-02, ...,
        3.34448161e-03, 3.34448161e-03, 3.34448161e-03],
       [1.38777878e-16, 0.00000000e+00, 5.20417043e-18, ...,
        1.73472348e-18, 1.73472348e-18, 1.73472348e-18],
       [1.24900090e-16, 0.00000000e+00, 4.33680869e-18, ...,
        2.16840434e-18, 2.16840434e-18, 2.16840434e-18]])

## Conclusions:
1. From cluster 1 to cluster 3, addition of second word from column 'topics' has no effect on the cluster centers, indicating the negligible effect of words from 'topics' column on cluster centers.
2. From cluster 2 to cluster3 , there is a change in only one cluster center, indicating the effect of additional words from 'keywords' and 'groups' column on cluster centers.
3. Further analysis can be done which details of effect of each subsequent word from all the three columns on cluster centers.